In [73]:
#import libraries

import os
import numpy as np
import csv
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import torch
import math
import time
import re
from nltk.corpus import stopwords
import nltk
from tqdm import tqdm
from transformers import pipeline
from tqdm import tqdm

In [78]:
stock_name = 'TSLA'

all_tweets = pd.read_csv('data/stock_tweets.csv')
print(all_tweets.shape)
all_tweets.head()
df = all_tweets[all_tweets['Stock Name'] == 'TSLA']
#all_tweets

(80793, 4)


In [79]:


nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def clean_text(text):
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'@\w+', '', text)     # Remove mentions
    text = re.sub(r'[^A-Za-z0-9\s]', '', text)  # Remove special characters
    text = text.lower()  # Convert to lowercase
    text = ' '.join([word for word in text.split() if word not in stop_words])  # Remove stop words
    text = text.strip()  # Remove extra whitespace
    return text

# Load your dataset


# Apply cleaning
df.loc[:, 'cleaned_text'] = df['Tweet'].apply(clean_text)

# Display the cleaned text
print(df[['Tweet', 'cleaned_text']].head())

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Haichen\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


                                               Tweet  \
0  Mainstream media has done an amazing job at br...   
1  Tesla delivery estimates are at around 364k fr...   
2  3/ Even if I include 63.0M unvested RSUs as of...   
3  @RealDanODowd @WholeMarsBlog @Tesla Hahaha why...   
4  @RealDanODowd @Tesla Stop trying to kill kids,...   

                                        cleaned_text  
0  mainstream media done amazing job brainwashing...  
1  tesla delivery estimates around 364k analysts ...  
2  3 even include 630m unvested rsus 630 addition...  
3  hahaha still trying stop tesla fsd bro get shi...  
4         stop trying kill kids sad deranged old man  


C:\Users\Haichen\AppData\Local\Temp\ipykernel_25784\1156406795.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'cleaned_text'] = df['Tweet'].apply(clean_text)


In [6]:
from transformers import pipeline
import pandas as pd

# Check for GPU availability
device = 0 if torch.cuda.is_available() else -1

# Initialize the sentiment analysis pipeline with GPU support
sentiment_pipeline = pipeline("sentiment-analysis", 
                              model="distilbert/distilbert-base-uncased-finetuned-sst-2-english",
                              device=device)

# Apply the sentiment analysis pipeline to the 'cleaned_text' column
df['Sentiment'] = df['cleaned_text'].apply(lambda x: sentiment_pipeline(x)[0]['label'])
df['Sentiment Score'] = df['cleaned_text'].apply(lambda x: sentiment_pipeline(x)[0]['score'])



C:\Users\Haichen\AppData\Local\Temp\ipykernel_25784\224264274.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Sentiment'] = df['cleaned_text'].apply(lambda x: sentiment_pipeline(x)[0]['label'])
C:\Users\Haichen\AppData\Local\Temp\ipykernel_25784\224264274.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Sentiment Score'] = df['cleaned_text'].apply(lambda x: sentiment_pipeline(x)[0]['score'])


Here I was trying to creat a csv that held the sentiment analysis for ten days, that way we could predict stock price. Do you think

> Add blockquote



In [22]:


# Check for GPU availability
device = 0 if torch.cuda.is_available() else -1

# Load your DataFrame
df = pd.read_csv('sentiment_1.csv')

# Initialize the zero-shot classification pipeline
classifier = pipeline('zero-shot-classification', 
                      model='facebook/bart-large-mnli', 
                      device=device)

# Define candidate labels
candidate_labels = [
    "finance", "automotive", "energy", "politics", "science",
    "travel", "lifestyle", "environment", "nonsense"
]

# Initialize tqdm for pandas
tqdm.pandas()

# Apply zero-shot classification with progress bar
def classify_text(text):
    if text.strip():
        result = classifier(text, candidate_labels=candidate_labels)
        scores = {label: score for label, score in zip(result['labels'], result['scores'])}
        return scores
    return {label: 0.0 for label in candidate_labels}

# Use tqdm to apply the classification with a progress bar
classification_results = df['cleaned_text'].progress_apply(classify_text)
classification_df = pd.DataFrame(classification_results.tolist(), index=df.index)

# Merge the classification results with the original DataFrame
df = df.join(classification_df)

# Save the DataFrame to a CSV file
df.to_csv('sentiment_analysis_results.csv', index=False)


100%|██████████| 37412/37412 [9:15:36<00:00,  1.12it/s]        


In [25]:
df.describe()

,Sentiment Score,automotive,nonsense,energy,lifestyle,environment,travel,finance,science,politics
count,37412.000000,37412.000000,37412.000000,37412.000000,37412.000000,37412.000000,37412.000000,37412.000000,37412.000000,37412.000000
mean,0.949543,0.147769,0.088471,0.113572,0.113697,0.131877,0.145464,0.150157,0.057432,0.051561
std,0.097226,0.206521,0.153946,0.128452,0.094370,0.095335,0.143958,0.162178,0.055567,0.074633
min,0.500692,0.000994,0.000507,0.000972,0.001158,0.001171,0.001047,0.000725,0.000434,0.000324
25%,0.960972,0.045565,0.016866,0.048259,0.054969,0.070147,0.062494,0.037281,0.024338,0.017060
50%,0.989832,0.073172,0.033204,0.081414,0.095991,0.121001,0.106193,0.083793,0.046837,0.032875
75%,0.996992,0.119356,0.074600,0.125635,0.141932,0.170767,0.167940,0.216086,0.073608,0.056396
max,0.999892,0.988521,0.987477,0.980004,0.953745,0.967888,0.975081,0.974547,0.914328,0.974999


In [7]:
import pandas as pd

df = pd.read_csv('sentiment_1.csv')
df_stock_price = pd.read_csv('data\stock_yfinance_data.csv')

df['Stock Name'] = df['Stock Name'].str.upper().str.strip()
df_stock_price['Stock Name'] = df_stock_price['Stock Name'].str.upper().str.strip()
# Convert 'Date' columns to datetime if not already, and remove timezone if present

df_stock_price = df_stock_price[df_stock_price['Stock Name'] == stock_name ]

df['Date'] = pd.to_datetime(df['Date']).dt.date
df_stock_price['Date'] = pd.to_datetime(df_stock_price['Date']).dt.date
def get_tech_ind(data):
    data['MA7'] = data['Close'].rolling(window=7).mean()
    data['MA20'] = data['Close'].rolling(window=20).mean()
    data['MACD'] = data['Close'].ewm(span=26).mean() - data['Open'].ewm(span=12, adjust=False).mean()
    data['20SD'] = data['Close'].rolling(window=20).std()
    data['upper_band'] = data['MA20'] + (data['20SD'] * 2)
    data['lower_band'] = data['MA20'] - (data['20SD'] * 2)
    data['EMA'] = data['Close'].ewm(com=0.5).mean()
    data['logmomentum'] = np.log(data['Close'] / data['Close'].shift(1))
    return data
label_df = get_tech_ind(df_stock_price)

nearest_trading_days = label_df['Date'].sort_values().reset_index(drop=True)

# Map each non-trading day to the nearest trading day
df['Nearest Trading Day'] = df['Date'].apply(lambda x: nearest_trading_days[nearest_trading_days <= x].max())

# Merge tweet data onto stock price data using the 'Nearest Trading Day'
final_df = pd.merge(df,label_df , left_on='Nearest Trading Day', right_on='Date', how='left')


NameError: name 'stock_name' is not defined

In [67]:
num_open_none = final_df['Open'].isna().sum()

print(f"Number of entries where 'Open' is None: {num_open_none}")
final_df.to_csv("final_data.csv")

Number of entries where 'Open' is None: 0
